In [1]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.orm import Session

# import and establish Base for which classes will be constructed
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

# import modlues to declare columns and column data types
from sqlalchemy import Column, Integer, String, Float

In [2]:
# read in netflix data downloaded from 
netflix = pd.read_csv("../../cars_in_movies_project/output_data/netflix/netflix_titles.csv")
netflix

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...
...,...,...,...,...,...,...,...,...,...,...,...,...
7782,s7783,Movie,Zozo,Josef Fares,"Imad Creidi, Antoinette Turk, Elias Gergi, Car...","Sweden, Czech Republic, United Kingdom, Denmar...","October 19, 2020",2005,TV-MA,99 min,"Dramas, International Movies",When Lebanon's Civil War deprives Zozo of his ...
7783,s7784,Movie,Zubaan,Mozez Singh,"Vicky Kaushal, Sarah-Jane Dias, Raaghav Chanan...",India,"March 2, 2019",2015,TV-14,111 min,"Dramas, International Movies, Music & Musicals",A scrappy but poor boy worms his way into a ty...
7784,s7785,Movie,Zulu Man in Japan,NaN,Nasty C,NaN,"September 25, 2020",2019,TV-MA,44 min,"Documentaries, International Movies, Music & M...","In this documentary, South African rapper Nast..."
7785,s7786,TV Show,Zumbo's Just Desserts,NaN,"Adriano Zumbo, Rachel Khoo",Australia,"October 31, 2020",2019,TV-PG,1 Season,"International TV Shows, Reality TV",Dessert wizard Adriano Zumbo looks for the nex...


In [3]:
# check data types for the CSV dataset
netflix.dtypes

show_id         object
type            object
title           object
director        object
cast            object
country         object
date_added      object
release_year     int64
rating          object
duration        object
listed_in       object
description     object
dtype: object

In [4]:
# create the 'Netflix' class
class Netflix(Base):
    __tablename__ = 'netflix_titles'
    id = Column(Integer, primary_key=True)
    show_id = Column(String(255))
    type_ = Column(String(255))
    title = Column(String(255))
    director = Column(String(255))
    cast = Column(String(255))
    country = Column(String(255))
    date_added = Column(String(255))
    release_year = Column(Integer)
    rating = Column(String(255))
    duration = Column(String(255))
    listed_in = Column(String(255))
    description = Column(String(255))

In [5]:
# create a connection to a SQLite database
engine = create_engine("sqlite:///resources/netflix_db.sqlite")

In [6]:
# create the 'netflix_titles' table withing the database
Base.metadata.create_all(engine)

In [7]:
# to push the objects made and query the server we use a Session object
session = Session(bind=engine)

In [8]:
# iterate over each row
for index, row in netflix.iterrows():
    # create an instance of the 'Netflix' class
    netflix_instance = Netflix(show_id=row[0], type_=row[1], title=row[2], director=row[3], cast=row[4],
                               country=row[5], date_added=row[6], release_year=row[7], rating=row[8], duration=row[9],
                               listed_in=row[10], description=row[11])
    # add object to the session
    session.add(netflix_instance)

# commit the objects to the database
session.commit()

In [9]:
# verification: query some of the items and print their information
items = session.query(Netflix)
for item in items:
    if item.id >= 11:
        break
    else:
        print("-"*12)
        print(f'id: {item.id}')
        print(f'title: {item.title}')
        print(f'director: {item.director}')
        print(f'cast: {item.cast}')
        print(f'release year: {item.release_year}')
        print(f'rating: {item.rating}')

# close the session
session.close()

------------
id: 1
title: 3%
director: None
cast: João Miguel, Bianca Comparato, Michel Gomes, Rodolfo Valente, Vaneza Oliveira, Rafael Lozano, Viviane Porto, Mel Fronckowiak, Sergio Mamberti, Zezé Motta, Celso Frateschi
release year: 2020
rating: TV-MA
------------
id: 2
title: 7:19
director: Jorge Michel Grau
cast: Demián Bichir, Héctor Bonilla, Oscar Serrano, Azalia Ortiz, Octavio Michel, Carmen Beato
release year: 2016
rating: TV-MA
------------
id: 3
title: 23:59
director: Gilbert Chan
cast: Tedd Chan, Stella Chung, Henley Hii, Lawrence Koh, Tommy Kuan, Josh Lai, Mark Lee, Susan Leong, Benjamin Lim
release year: 2011
rating: R
------------
id: 4
title: 9
director: Shane Acker
cast: Elijah Wood, John C. Reilly, Jennifer Connelly, Christopher Plummer, Crispin Glover, Martin Landau, Fred Tatasciore, Alan Oppenheimer, Tom Kane
release year: 2009
rating: PG-13
------------
id: 5
title: 21
director: Robert Luketic
cast: Jim Sturgess, Kevin Spacey, Kate Bosworth, Aaron Yoo, Liza Lapira, 